<a href="https://colab.research.google.com/github/ktam069/Google_Colab_Audiovisual/blob/master/Audiovisual_basic_TFServing_on_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls '/content/gdrive/My Drive/Part IV Project/Colab Notebooks/dataset_npy/'

dataset_train_0-20.npy


In [0]:
!cp '/content/gdrive/My Drive/Part IV Project/Colab Notebooks/dataset_npy/dataset_train_0-20.npy' "dataset_train_0-20.npy"

In [0]:
!ls

dataset_train_0-20.npy	gdrive	sample_data


In [0]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print(tf.__version__)

1.14.0


In [0]:
dataset_train = np.load("dataset_train_0-20.npy")

In [0]:
# Split dataset into inputs and ground truths
x_train = dataset_train[:,:,:,:,0 ]
y_train = dataset_train[:,:,:,:,1:]

In [0]:
print(x_train.shape)
print(y_train.shape)

(210, 298, 257, 2)
(210, 298, 257, 2, 2)


In [0]:
!cp '/content/gdrive/My Drive/Part IV Project/Colab TFServing/saved_model_basic_22_09_045241.h5' "saved_model_basic_22_09_045241.h5"

In [0]:
from tensorflow.keras.models import load_model
model = load_model("saved_model_basic_22_09_045241.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Most of the following code in this notebook was adapted from https://www.tensorflow.org/tfx/serving/tutorials/Serving_REST_simple

In [0]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))
if os.path.isdir(export_path):
  print('\nAlready saved a model, cleaning up\n')
  !rm -r {export_path}

tf.saved_model.simple_save(
    keras.backend.get_session(),
    export_path,
    inputs={'input_image': model.input},
    outputs={t.name:t for t in model.outputs})

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/1/saved_model.pb

Saved model:
total 604
-rw-r--r-- 1 root root 613190 Sep 25 13:46 saved_model.pb
drwxr-xr-x 2 root root   4096 Sep 25 13:46 variables


In [0]:
'''
MODEL_DIR = ""
version = 1
export_path = os.path.join(MODEL_DIR, str(version))

print('export_path = {}\n'.format(export_path))
if os.path.isdir(export_path):
  print('Already saved a model, cleaning up\n')
  !rm -r {export_path}

tf.contrib.saved_model.save_keras_model(model, export_path)

#!mkdir {export_path}
!ls
'''

'\nMODEL_DIR = ""\nversion = 1\nexport_path = os.path.join(MODEL_DIR, str(version))\n\nprint(\'export_path = {}\n\'.format(export_path))\nif os.path.isdir(export_path):\n  print(\'Already saved a model, cleaning up\n\')\n  !rm -r {export_path}\n\ntf.contrib.saved_model.save_keras_model(model, export_path)\n\n#!mkdir {export_path}\n!ls\n'

In [0]:
print('\nSaved model:')
!ls -l {export_path}


Saved model:
total 604
-rw-r--r-- 1 root root 613190 Sep 25 13:46 saved_model.pb
drwxr-xr-x 2 root root   4096 Sep 25 13:46 variables


In [0]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_image'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 298, 257, 2)
        name: input_layer_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_layer/Reshape:0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 298, 257, 2, 2)
        name: output_layer/Reshape:0
  Method name is: tensorflow/serving/predict


In [0]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   110k      0 --:--:-- --:--:-- --:--:--  110k
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:3 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server-universal amd64 Packages [365 B]
Get:4 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 Packages [357 B]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [70.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/m

In [0]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 124 not upgraded.
Need to get 151 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 1.14.0 [151 MB]
Fetched 151 MB in 2s (61.2 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_1.14.0_all.deb ...
Unpacking tensorflow-model-server (1.14.0) ...
Setting up tensorflow-model-server (1.14.0) ...


In [0]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [0]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=avspeech_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


In [0]:
!tail server.log

2019-09-25 13:47:09.123330: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:54] Reading meta graph with tags { serve }
2019-09-25 13:47:09.130721: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-09-25 13:47:09.175564: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:202] Restoring SavedModel bundle.
2019-09-25 13:47:09.262786: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:311] SavedModel load for tags { serve }; Status: success. Took 147107 microseconds.
2019-09-25 13:47:09.262890: I tensorflow_serving/servables/tensorflow/saved_model_warmup.cc:103] No warmup data file found at /tmp/1/assets.extra/tf_serving_warmup_requests
2019-09-25 13:47:09.262989: I tensorflow_serving/core/loader_harness.cc:86] Successfully loaded servable version {name: avspeech_model version: 1}
2019-09-25 13:47:09.264948: I tensorflow_serving/mo

In [0]:
'''
def show(idx, title):
  plt.figure()
  plt.imshow(test_images[idx].reshape(28,28))
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

import random
rando = random.randint(0,len(test_images)-1)
show(rando, 'An Example Image: {}'.format(class_names[test_labels[rando]]))
'''

"\ndef show(idx, title):\n  plt.figure()\n  plt.imshow(test_images[idx].reshape(28,28))\n  plt.axis('off')\n  plt.title('\n\n{}'.format(title), fontdict={'size': 16})\n\nimport random\nrando = random.randint(0,len(test_images)-1)\nshow(rando, 'An Example Image: {}'.format(class_names[test_labels[rando]]))\n"

In [0]:
!pip install -q requests

In [0]:
test_images = x_train

In [0]:
test_images[0:1].shape

(1, 298, 257, 2)

In [0]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:1].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

Data: {"signature_name": "serving_default", "instances": ...  0.28270596265792847], [0.2902107834815979, 0.0]]]]}


In [0]:
import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/avspeech_model:predict', data=data, headers=headers)

In [0]:
predictions = json.loads(json_response.text)['predictions']

In [0]:
print("result string length:", len(json_response.text))
print(json_response.text[:128])

result string length: 4371305
{
    "predictions": [[[[[0.50045526, 0.499316454], [0.49955678, 0.499397397]], [[0.500694752, 0.499889225], [0.499602556, 0.499


In [0]:
print(len(predictions))
print(str(predictions)[:100])
#print(predictions)

1
[[[[[0.50045526, 0.499316454], [0.49955678, 0.499397397]], [[0.500694752, 0.499889225], [0.499602556


In [0]:
import ast
results = ast.literal_eval(str(predictions))
results = np.array(results)
print(results.shape)
print(results[0,:1,:3])

(1, 298, 257, 2, 2)
[[[[0.50045526 0.49931645]
   [0.49955678 0.4993974 ]]

  [[0.50069475 0.49988922]
   [0.49960256 0.49962071]]

  [[0.50083208 0.49971414]
   [0.49978095 0.49952585]]]]


In [0]:
np.save("avspeech_output", results)

In [0]:
new_dir = "/content/gdrive/My Drive/Part IV Project/Colab TFServing/AV_basic_output/"
if not os.path.isdir(new_dir):
  os.mkdir(new_dir)

In [0]:
!cp "avspeech_output.npy" "/content/gdrive/My Drive/Part IV Project/Colab TFServing/AV_basic_output/avspeech_output.npy"

In [0]:
# Call a particular version
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/avspeech_model/versions/1:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

In [0]:
import time

In [0]:
headers = {"content-type": "application/json"}
start_time = time.time()
json_response = requests.post('http://localhost:8501/v1/models/avspeech_model:predict', data=data, headers=headers)
end_time = time.time()
print(end_time-start_time)

9.772232294082642


In [0]:
headers = {"content-type": "application/json"}

times = []
num_tests = 30

for i in range(num_tests):
  d = json.dumps({"signature_name": "serving_default", "instances": test_images[i:i+1].tolist()})
  start_time = time.time()
  json_response = requests.post('http://localhost:8501/v1/models/avspeech_model:predict', data=d, headers=headers)
  end_time = time.time()
  runtime = end_time-start_time
  times.append(runtime)

avg = sum(times) / len(times)
print("Number of test:", len(times))
print("Test times:", times)
print("Average running time:", avg)

Number of test: 30
Test times: [9.687697887420654, 9.431162118911743, 9.37843108177185, 9.368245363235474, 9.30911922454834, 9.307308912277222, 9.366823196411133, 9.367759227752686, 9.529803037643433, 9.526405096054077, 9.49414873123169, 9.576581239700317, 9.372554063796997, 9.297653436660767, 9.367451667785645, 9.394316673278809, 9.325661897659302, 9.46376085281372, 9.43920350074768, 9.438074111938477, 9.63651704788208, 9.44785475730896, 9.390293598175049, 9.258939504623413, 9.33358383178711, 9.422390460968018, 9.68515157699585, 9.66890811920166, 9.642473459243774, 9.656212568283081]
Average running time: 9.452816208203634


In [0]:
headers = {"content-type": "application/json"}

times = []
num_tests = 30

for i in range(num_tests):
  d = json.dumps({"signature_name": "serving_default", "instances": test_images[i:i+1].tolist()})
  start_time = time.time()
  json_response = requests.post('http://localhost:8501/v1/models/avspeech_model:predict', data=d, headers=headers)
  end_time = time.time()
  runtime = end_time-start_time
  times.append(runtime)

avg = sum(times) / len(times)
print("Number of test:", len(times))
print("Test times:", times)
print("Average running time:", avg)

Number of test: 30
Test times: [9.488592624664307, 9.57523226737976, 9.638319730758667, 9.626328945159912, 9.680450201034546, 9.574971675872803, 9.52237868309021, 9.452747344970703, 9.417555570602417, 9.454872608184814, 9.54455041885376, 9.556065797805786, 9.486701011657715, 9.44108533859253, 9.43125319480896, 9.62403917312622, 9.797571897506714, 9.750877380371094, 9.692318439483643, 9.65929365158081, 9.738107204437256, 9.853883266448975, 9.773972034454346, 9.716116428375244, 9.809432983398438, 9.835650205612183, 9.845261096954346, 10.029192924499512, 9.808818578720093, 9.81845235824585]
Average running time: 9.65480310122172


In [0]:
headers = {"content-type": "application/json"}

times = []
num_tests = 30

for i in range(num_tests):
  d = json.dumps({"signature_name": "serving_default", "instances": test_images[i:i+1].tolist()})
  start_time = time.time()
  json_response = requests.post('http://localhost:8501/v1/models/avspeech_model:predict', data=d, headers=headers)
  end_time = time.time()
  runtime = end_time-start_time
  times.append(runtime)

avg = sum(times) / len(times)
print("Number of test:", len(times))
print("Test times:", times)
print("Average running time:", avg)

Number of test: 30
Test times: [5.875006914138794, 5.7138895988464355, 5.669135570526123, 5.598212957382202, 5.440412521362305, 5.518384218215942, 5.378460168838501, 5.528931617736816, 5.41772985458374, 5.416262865066528, 5.43831467628479, 5.454464912414551, 5.457348346710205, 5.533688545227051, 5.487878084182739, 5.217731714248657, 5.232433557510376, 5.152238130569458, 5.19244909286499, 5.18646502494812, 5.1934897899627686, 5.181354761123657, 5.2429211139678955, 5.198957443237305, 5.199333190917969, 5.063434600830078, 5.0766355991363525, 5.056780576705933, 5.206295490264893, 5.236968994140625]
Average running time: 5.352186997731526


In [0]:
headers = {"content-type": "application/json"}

times = []
num_tests = 100

for i in range(num_tests):
  d = json.dumps({"signature_name": "serving_default", "instances": test_images[i:i+1].tolist()})
  start_time = time.time()
  json_response = requests.post('http://localhost:8501/v1/models/avspeech_model:predict', data=d, headers=headers)
  end_time = time.time()
  runtime = end_time-start_time
  times.append(runtime)

avg = sum(times) / len(times)
print("Number of test:", len(times))
print("Test times:", times)
print("Average running time:", avg)

Number of test: 100
Test times: [5.278883218765259, 5.383906602859497, 5.351484537124634, 5.3050994873046875, 5.249265432357788, 5.348854303359985, 5.262177228927612, 5.2690465450286865, 5.374250650405884, 5.533603191375732, 5.498345613479614, 5.26525354385376, 5.230463027954102, 5.29756236076355, 5.231868028640747, 5.1011247634887695, 5.0682783126831055, 5.08689546585083, 5.097135305404663, 5.170293807983398, 5.11630916595459, 5.089215040206909, 5.1259987354278564, 5.090400695800781, 5.1059958934783936, 5.156183242797852, 5.120934963226318, 5.049148321151733, 5.055005311965942, 4.997024297714233, 5.011581659317017, 5.058854103088379, 4.964389085769653, 4.978443622589111, 4.9912941455841064, 5.018040895462036, 4.9677653312683105, 5.01768946647644, 4.96070408821106, 5.026818752288818, 5.088963508605957, 5.101289510726929, 5.048648118972778, 4.950058221817017, 4.824756860733032, 4.940420866012573, 4.8930535316467285, 4.929257154464722, 5.005452632904053, 4.942628383636475, 4.965582847595

In [0]:
headers = {"content-type": "application/json"}

times = []
num_tests = 100

for i in range(num_tests):
  d = json.dumps({"signature_name": "serving_default", "instances": test_images[i:i+1].tolist()})
  start_time = time.time()
  json_response = requests.post('http://localhost:8501/v1/models/avspeech_model:predict', data=d, headers=headers)
  end_time = time.time()
  runtime = end_time-start_time
  times.append(runtime)

avg = sum(times) / len(times)
print("Number of test:", len(times))
print("Test times:", times)
print("Average running time:", avg)

Number of test: 100
Test times: [5.81235671043396, 5.713325500488281, 6.416647434234619, 5.840157508850098, 6.1336586475372314, 6.357832193374634, 6.408709764480591, 6.46977424621582, 6.409029006958008, 6.185210227966309, 6.001291275024414, 6.027223348617554, 5.966468095779419, 5.9876837730407715, 5.878033638000488, 5.82472562789917, 5.913345813751221, 5.924131870269775, 6.035085678100586, 6.02473783493042, 6.026598691940308, 5.927107095718384, 5.916055679321289, 5.862263917922974, 5.9963085651397705, 6.1368184089660645, 5.916751861572266, 6.134869813919067, 6.529178857803345, 6.084124565124512, 5.874738693237305, 6.044142961502075, 6.009650707244873, 5.992861032485962, 5.915069818496704, 5.862044811248779, 5.88100790977478, 5.858445882797241, 6.0270020961761475, 5.814558267593384, 5.865767002105713, 5.756867170333862, 5.776489019393921, 5.754576206207275, 5.633662700653076, 5.5552146434783936, 5.771476745605469, 5.631785154342651, 5.783599853515625, 5.586092948913574, 5.74322223663330

In [0]:
np.savetxt("TFServing_GPU_times.csv", times, delimiter=",")

In [0]:
!cp "TFServing_GPU_times.csv" "/content/gdrive/My Drive/Part IV Project/Colab TFServing/AV_basic_output/TFServing_GPU_times.csv"